In [2]:
using Pkg
Pkg.activate(".")
using BenchmarkTools
import Markdown 
Base.showable(::MIME"text/markdown", ::Markdown.MD) =true # same function as below
include("bnc_initialize.jl")

  Activating project at `/mnt/d/Realizibility-index/bnc_julia`


show_qK_space_constrains (generic function with 1 method)

In [4]:
L = [1 0 1 3
    0 1 1  1]
model = Bnc(L=L)
[model.L
model.N]

4×4 Matrix{Int64}:
 1  0   1   3
 0  1   1   1
 1  1  -1   0
 3  1   0  -1

We especially care about while it will form: 
$$\begin{bmatrix}
0&0&0&1\\
0&0&1&0
\end{bmatrix}$$
$$\begin{bmatrix}
0&0&y&x\\
0&0&z&0
\end{bmatrix}$$

In [5]:
logK1 = 0
logK2_rg = (-6,6)
logE_rg = (-6,6)
logS_rg = (-6,6)

total_points = 6000000
point_num_eachdim = round(Int, total_points^(1/3))

#idx1: logE_min ~logE_max
#idx2: logS_min ~logS_max
#idx3: logK2_min ~logK2_max

#Initialize the array
logx_mt = Array{Vector{Float64},3}(undef, point_num_eachdim, point_num_eachdim, point_num_eachdim)

#Fill the first axis
start_logqK = [logE_rg[1], logS_rg[1], logK1 , logK2_rg[1]]
end_logqK = [logE_rg[2], logS_rg[1], logK1 , logK2_rg[1]]
logx_mt[:,1,1] .= x_traj_with_qK_change(model,start_logqK, end_logqK;input_logspace=true, output_logspace=true,saveat = range(0,1,length=point_num_eachdim))[2]

#Fill the first surface
logE_step = range(logE_rg...,length=point_num_eachdim)
for i in 1:point_num_eachdim
    start_logqK = [logE_step[i], logS_rg[1], logK1 , logK2_rg[1]]
    end_logqK = [logE_step[i], logS_rg[2], logK1 , logK2_rg[1]]
    logx_mt[i,:,1] .= x_traj_with_qK_change(model, start_logqK, end_logqK; startlogx = logx_mt[i,1,1],input_logspace=true, output_logspace=true ,saveat = range(0,1,length=point_num_eachdim))[2]
end

#Fill the whole cube
logS_step = range(logS_rg...,length=point_num_eachdim)
for i in 1:point_num_eachdim
    for j in 1:point_num_eachdim
        start_logqK = [logE_step[i], logS_step[j], logK1 , logK2_rg[1]]
        end_logqK = [logE_step[i], logS_step[j], logK1 , logK2_rg[2]]
        logx_mt[i,j,:] .= x_traj_with_qK_change(model, start_logqK, end_logqK; startlogx = logx_mt[i,j,1],input_logspace=true, output_logspace=true ,saveat = range(0,1,length=point_num_eachdim))[2]
    end
end


In [4]:
x_rgm_asym = logx_mt .|> x-> assign_vertex_x(model,x,input_logspace=true,asymtotic=true) |> x->get_idx(model,x)
qK_rgm_asym = logx_mt .|> x-> assign_vertex_qK(model,x,input_logspace=true,asymtotic=true) |> x->get_idx(model,x)

Start finding all vertices, it may takes a while.
Done, with 8 vertices found and 7 real vertices.


182×182×182 Array{Int64, 3}:
[:, :, 1] =
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1  1  1  1  1  1     4  4  4  4  4  4  4  4  4  4  4  4
 1  1  1  1  1  1  1  1

In [23]:
log10.([0.7477183827605597, 0.17364247976708896, 0.12983567414998107, 0.06725444326766648])

4-element Vector{Float64}:
 -0.12626194203175528
 -0.7603440206824899
 -0.8866059627142449
 -1.1722790180484701

In [25]:
assign_vertex_qK(model,[0.7477183827605597, 0.17364247976708896, 0.12983567414998107, 0.06725444326766648],input_logspace=true,asymtotic=false)

minimum(C * logqK .+ C0) = -0.7682598363864444
minimum(C * logqK .+ C0) = -0.540881557645253
minimum(C * logqK .+ C0) = -0.061092619810969084
minimum(C * logqK .+ C0) = -1.3342109048889508
minimum(C * logqK .+ C0) = 0.028708333387623464


2-element Vector{Int8}:
 4
 2

In [11]:
assign_vertex_qK(model,[0.7477183827605597, 0.17364247976708896, 0.12983567414998107, 0.06725444326766648],input_logspace=true,asymtotic=false)

2-element Vector{Int8}:
 4
 2

In [42]:
assign_vertex_qK(model,[0.8376461557570573, 0.18735713970059872, 0.15693898782384472, 0.08757991773399734],input_logspace=false,asymtotic=false)

minimum(C * logqK .+ C0) = -0.21192788728645656
minimum(C * logqK .+ C0) = -0.11248037201261524
minimum(C * logqK .+ C0) = -0.013032855277088029
minimum(C * logqK .+ C0) = -1.3058505397926556
minimum(C * logqK .+ C0) = -0.028804886178355832


┌ Error: No finite neighbor found for [0.8376461557570573, 0.18735713970059872, 0.15693898782384472, 0.08757991773399734]
└ @ Main /mnt/d/Realizibility-index/bnc_julia/bnc_numeric.jl:684


In [3]:
x_rgm = logx_mt .|> x-> assign_vertex_x(model,x,input_logspace=true,asymtotic=false) |> x->get_idx(model,x)
qK_rgm = logx_mt .|> x-> assign_vertex_qK(model,x,input_logspace=true,asymtotic=false,eps=0.015) |> x->get_idx(model,x)

logK2_step = range(logK2_rg...,length=point_num_eachdim)
(minval,maxval) = values(model.vertices_idx) |> x->(minimum(x),maximum(x))

LoadError: UndefVarError: `logx_mt` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
Set(x_rgm[:])|> collect|>sort

In [29]:
Set(qK_rgm[:]) |>collect |>sort

LoadError: MethodError: no method matching isless(::Nothing, ::Vector{Int8})
The function `isless` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  isless([91m::Missing[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:87[24m[39m
[0m  isless(::Any, [91m::Missing[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:88[24m[39m
[0m  isless([91m::AbstractVector[39m, ::AbstractVector)
[0m[90m   @[39m [90mBase[39m [90m[4mabstractarray.jl:3026[24m[39m
[0m  ...


In [ ]:
idx = 3
perm = get_perm(model,idx)
is_real = get_vertex!(model,idx).real
singularity = get_singularity!(model,idx)
println("idx=$idx,perm=$perm, is_real=$is_real, singularity=$singularity")

Start calculating vertex neighbor matrix, It may takes a while.Done.idx=3,perm=Int8[1, 4], is_real=true, singularity=0


In [ ]:
using GLMakie

# 参数
nslices = 11   # 可以随便改 slice 数目
slice_positions = round.(Int, range(1, point_num_eachdim, length=nslices+2)[2:end-1])

f = Figure(size=(600*nslices, 2000))

cmap = :rainbow
nlevels = maxval - minval + 1
cmap_disc = cgrad(cmap, nlevels, categorical=true)
crange = (minval, maxval)

# --- 3D 图 ---
p11 = Axis3(f[1, 1])
p11.title = "x space non-asym"
volume!(p11, x_rgm; colormap=cmap, colorrange=crange)

p12 = Axis3(f[2, 1])
p12.title = "qK space non-asym"
volume!(p12, qK_rgm; colormap=cmap, colorrange=crange)

p13 = Axis3(f[3, 1])
p13.title = "x space asym"
volume!(p13, x_rgm_asym; colormap=cmap, colorrange=crange)

p14 = Axis3(f[4, 1])
p14.title = "qK space asym"
volume!(p14, qK_rgm_asym; colormap=cmap, colorrange=crange)

# --- slices 横向排列 ---
for (i, idx) in enumerate(slice_positions)
    # x space slice
    ax = Axis(f[1, i+1])
    ax.title = "x slice $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax, logE_step, logS_step, x_rgm[:, :, idx]; colormap=cmap, colorrange=crange)

    # qK space slice
    ax2 = Axis(f[2, i+1])
    ax2.title = "qK slice $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax2, logE_step, logS_step, qK_rgm[:, :, idx]; colormap=cmap, colorrange=crange)

    ax3 = Axis(f[3, i+1])
    ax3.title = "x slice asym $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax3, logE_step, logS_step, x_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)

    ax4 = Axis(f[4, i+1])
    ax4.title = "qK slice asym $i,with logK2 = $(round(logK2_step[idx],digits=2))"
    heatmap!(ax4, logE_step, logS_step, qK_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)
end

# label
for p in [p11, p12]
    p.xlabel = "logE"
    p.ylabel = "logS"
    p.zlabel = "logK2"
end
# for ax in content(f)
#     if ax isa Axis
#         ax.xlabel = "logE"
#         ax.ylabel = "logS"
#     end
# end

# colorbar
Colorbar(f[:, end+1], colorrange=crange, colormap=cmap_disc, ticks=minval:maxval)
save("rgm_slices_and_volume.png", f)
display(f)


GLMakie.Screen(...)